In [1]:
# https://medium.com/@prasadmahamulkar/fine-tuning-phi-2-a-step-by-step-guide-e672e7f1d009
# Install and import the necessary libraries
# !pip install torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 
#  trl==0.4.7 accelerate einops 
## Issue with cuda size

# https://blog.gopenai.com/how-to-resolve-runtimeerror-cuda-out-of-memory-d48995452a0
# https://iamholumeedey007.medium.com/memory-management-using-pytorch-cuda-alloc-conf-dabe7adec130
# https://iamholumeedey007.medium.com/how-can-i-set-max-split-size-mb-to-avoid-fragmentation-in-pytorch-ff0a6c641098

# nvidia-smi
# taskkill /pid <pid> /f

## finally reducing batch size and gradient accumation steps from 4 to 1 
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from torch import cuda
cuda.is_available()
torch.cuda.empty_cache()
import gc
# del variables
gc.collect()

80

In [2]:
# Model
base_model = "microsoft/phi-2"
new_model = "phi-2-medquad"

# Dataset
dataset = load_dataset("prsdm/medquad-phi2-1k", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# LoRA configuration
peft_config = LoraConfig(
    r= 64,          
    lora_alpha= 16,
    lora_dropout=0.1, #0.05
    bias="none",
    task_type="CAUSAL_LM",
   #target_modules= ["Wqkv", "out_proj"] #["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ]
)

In [6]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 2, #4
    per_device_eval_batch_size = 2,  #4
    gradient_accumulation_steps = 2, #4
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)
# max_split_size_mb = 200
# allocator = torch.cuda.memory._get_memory_allocator()
# allocator.set_max_split_size(max_split_size_mb * 1024 * 1024)

max_split_size_mb = 180
allocator = torch.cuda.memory.max_memory_allocated()
# allocator.to_bytes()
# set 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1024'
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:200"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Train model
trainer.train()

C:\Users\HP\anaconda3\envs\kalichat\Lib\site-packages\trl\trainer\sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
C:\Users\HP\anaconda3\envs\kalichat\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
